In [1]:
using Graphs, SimpleWeightedGraphs
include("NetworkDistance.jl");

In [2]:
runtimes_baseline = Dict{Int64, Float64}(); # Dictionary "# nodes -> runtime in seconds" to store the running times of the baseline
runtime_solver = Dict{Int64, Float64}();    # Dictionary "# nodes -> runtime in seconds" to store the running times of the Laplacian solver

In [3]:
for n in [100, 200, 500, 1000, 2000, 5000]                                # For larger networks (n regulates the number of nodes)...
    for r in 0:1                                                          # ...make two runs (Julia's first run includes compilation time...
        G = adjacency_matrix(SimpleWeightedGraph(erdos_renyi(n, n * 2))); # ...make an Erdos-Renyi random graph with n node and avg degree of 4
        s = rand(Float64, n);                                             # Make a vector with a random value per node in the network
        t = rand(Float64, n);                                             # Make another random vector
        trial = @timed NetworkDistance.ge(G, s - t, "base");              # Time how much time it takes to estimate ge distance between s and t
        if r > 0                                                          # If this is not the first run...
            runtimes_baseline[n] = trial.time;                            # ...store the running time in the dictioanry
        end
    end
end

runtimes_baseline

Dict{Int64, Float64} with 6 entries:
  200  => 0.00552633
  500  => 0.0328485
  2000 => 1.1634
  5000 => 27.2497
  1000 => 0.159127
  100  => 0.00150724

In [4]:
# Repeat the experiment from the previous cell with the same code, but now using the "approxchol" Laplacian solver instead
for n in [100, 200, 500, 1000, 2000, 5000]
    for r in 0:1
        G = adjacency_matrix(SimpleWeightedGraph(erdos_renyi(n, n * 2)));
        s = rand(Float64, n);
        t = rand(Float64, n);
        trial = @timed NetworkDistance.ge(G, s - t, "approxchol");
        if r > 0
            runtime_solver[n] = trial.time;
        end
    end
end

runtime_solver

Dict{Int64, Float64} with 6 entries:
  200  => 0.000199472
  500  => 0.00042869
  2000 => 0.00248691
  5000 => 0.00735952
  1000 => 0.0010411
  100  => 0.0655915

In [5]:
n = 2000;                                                                # Set number of nodes to 2000

G = adjacency_matrix(SimpleWeightedGraph(erdos_renyi(n, n * 2)));        # Make a random graph with n nodes and avg degree of 4
s = rand(Float64, n);                                                    # Make a vector with a random value per node in the network
t = rand(Float64, n);                                                    # Make another

ge_base = NetworkDistance.ge(G, s - t, "base");                          # Calculate the exact value of ge with the baseline
ge_approx = NetworkDistance.ge(G, s - t, "approxchol");                  # Calculate the approximate value of ge with the "approxchol" Laplacian solver

println("Exact graph Euclidean distance: $(ge_base)");
println("Approximate graph Euclidean distance baseline: $(ge_approx)");

Exact graph Euclidean distance: 13.114699358420399
Approximate graph Euclidean distance baseline: 13.114699358417543
